In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\prajw\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [2]:
url = 'https://www.nike.com/in/w/sale-3yaep'  # nike sales page
driver.get(url)

def infinite_scroll():
    # get current height of the page
    height = driver.execute_script('return document.body.scrollHeight')
    while True:
        # scroll to end of the page
        driver.execute_script(f'window.scrollTo(0, document.body.scrollHeight)')

        # wait for 2 seconds so that the items in the page get loaded
        time.sleep(2)

        # find out the new height of the page after scrolling
        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == height:  # you have reached the end of the page, so break
            break

        # you have not yet reached the end of the page, so repeat untill you reach the end of the page
        height = new_height

# this function can be used anytime a page with infinite scrolling has to be scraped
infinite_scroll()

In [3]:
# now that the page is fully scrolled, scrape all details of all products
soup = BeautifulSoup(driver.page_source, parser='lxml')
products = soup.find_all('div', class_='product-card__body')

df = pd.DataFrame(columns=['Product name', 'Description',
                           'Colours', 'Price', 'Sale Price', 'Discount', 'Product Link'])

for product in products:
    # for some products,some data might be missing,so we skip that product
    try:
        name = product.find('div', class_='product-card__title').text.strip()
        description = product.find('div', class_='product-card__subtitle').text.strip()
        colours = product.find('div', class_='product-card__count-item').text.strip()
        price = product.find('div', class_='product-price css-1h0t5hy').text.strip()
        sale_price = product.find('div', class_='product-price is--current-price css-s56yt7').text.strip()

        # now we are going to calculate discount, as it is not specified on the page
        # remove '₹' and ',' from price and sales price and convert it to integer data type
        temp_price = int(price[1:].replace(',', ''))
        temp_sales_price = int(sale_price[1:].replace(',', ''))

        # calculate discount,round it up and add a '%' sign at the end
        discount = str(round((temp_price - temp_sales_price)/temp_price * 100)) + '%'

        link = product.find(
            'a', class_='product-card__link-overlay').get('href')

        # add all scraped info about one product to the dataframe, and move on to scrape info about the next product
        index = len(df)
        df.loc[index] = [name, description, colours,
                         price, sale_price, discount, link]
    except:
        pass

# store all data in a csv file
df.to_csv('nike sale data.csv', index=False)

In [4]:
len(df)  # number of product data scraped

313

In [5]:
df.head()

,Product name,Description,Colours,Price,Sale Price,Discount,Product Link
0,Nike Downshifter 10,Men's Running Shoe,1 Colour,"₹3,995","₹3,597",10%,https://www.nike.com/in/t/downshifter-10-runni...
1,Nike 50 Tech Fleece,Men's Jacket,1 Colour,"₹6,995","₹5,597",20%,https://www.nike.com/in/t/50-tech-fleece-jacke...
2,NikeCourt Blanc,Women's Shoe,2 Colours,"₹5,995","₹4,797",20%,https://www.nike.com/in/t/nikecourt-blanc-shoe...
3,Jordan Zoom Trunner Ultimate,Running Shoe,1 Colour,"₹12,995","₹10,397",20%,https://www.nike.com/in/t/jordan-zoom-trunner-...
4,Nike Joyride Dual Run,Men's Running Shoe,5 Colours,"₹10,995","₹7,697",30%,https://www.nike.com/in/t/joyride-dual-run-run...
